In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [2]:
df=pd.read_csv(r'dataset-final - 3000.csv')
df_copy=pd.read_csv(r'dataset-final - 3000.csv')

In [3]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Aycha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Aycha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Aycha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

**1. Prétraitement textuel avec NLTK**
Certains chiffres se rapportent à des catégories spécifiques (âge, quantité, année, etc.), On  les transformer en catégories spécifiques. Car le modele ne peut pas lire les chiffres.

In [4]:
import re
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()


def preprocess_text(text):
    if isinstance(text, str):  # Vérifie si le texte est une chaîne
        # Remplacer les chiffres par des catégories basées sur des expressions régulières
        text = re.sub(r'\b\d{1,2} ans\b', 'AGE', text)  # Remplacer les âges
        text = re.sub(r'\b\d{4}\b', 'YEAR', text)  # Remplacer les années
        text = re.sub(r'\b\d+\s?(kg|g|lbs|ounces|items|people|years|dollars|euro|hour|minutes)\b', 'QUANTITE', text)  # Remplacer les quantités

        # Tokenisation, Lemmatisation et suppression des stopwords
        tokens = word_tokenize(text.lower())  # Tokenisation
        tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalnum()]  # Lemmatisation
        tokens = [word for word in tokens if word not in stop_words]  # Suppression des stopwords
        
        return " ".join(tokens)
    return ""  # Retourne une chaîne vide si le texte n'est pas valide


df_copy["SYNOPSIS"] = df_copy["SYNOPSIS"].apply(preprocess_text)
df_copy["REVIEWS"] = df_copy["REVIEWS"].apply(preprocess_text)

Transformer la colonne GENRES en colonnes binaires, où chaque genre sera une nouvelle colonne contenant des valeurs 1 (présence du genre) ou 0 (absence du genre) 

In [5]:
df_copy['GENRES'] = df_copy['GENRES'].apply(lambda x: x.split('; ') if isinstance(x, str) else [])

# Créer une liste de tous les genres uniques présents dans la colonne GENRES
all_genres = set([genre for sublist in df_copy['GENRES'] for genre in sublist])

# Créer une nouvelle colonne pour chaque genre avec 1 ou 0 en fonction de la présence du genre
for genre in all_genres:
    df_copy[f'genre_{genre}'] = df_copy['GENRES'].apply(lambda x: 1 if genre in x else 0)


In [6]:
df_copy.columns

Index(['Unnamed: 0', 'MOVIE_NAME', 'REVIEWS', 'SYNOPSIS', 'YEAR', 'CAST',
       'DURATION_MIN', 'GENRES', 'PRODUCERS', 'RATING_SUR_5',
       'MAJORITY_SENTIMENT', 'MOST_FREQUENT_SENTIMENT', 'Rating_Category',
       'genre_War', 'genre_LGBTQ+', 'genre_Fantasy', 'genre_History',
       'genre_Holiday', 'genre_Documentary', 'genre_Mystery', 'genre_Music',
       'genre_Action', 'genre_Kids & Family', 'genre_Horror', 'genre_Anime',
       'genre_Musical', 'genre_Drama', 'genre_TV Movie', 'genre_Biography',
       'genre_Western', 'genre_Animation', 'genre_Sci-Fi', 'genre_Comedy',
       'genre_Thriller', 'genre_Adventure', 'genre_Sports', 'genre_Family',
       'genre_Crime', 'genre_Science Fiction', 'genre_Mystery & Thriller',
       'genre_Romance'],
      dtype='object')

Voir les acteurs les plus frequents dans les films les mieux notés 

In [7]:
# Assurer que chaque valeur dans 'CAST' est une liste
df_copy['CAST'] = df_copy['CAST'].apply(lambda x: x.split(', ') if isinstance(x, str) else [])
# Filtrer les entrées non pertinentes dans la colonne 'CAST' ('Show All…', 'Indisponible', '')
df_copy['CAST'] = df_copy['CAST'].apply(lambda x: [actor for actor in x if actor not in ['Show All…', 'Indisponible', '']])

# Filtrer les films avec une note supérieure à 3.5
good_movies = df_copy[df_copy['RATING_SUR_5'] >= 3.5]

# Aplatir la liste des acteurs pour les films ayant une note > 3.5
all_casts = [actor for sublist in good_movies['CAST'] for actor in sublist]

# Compter la fréquence de chaque acteur
from collections import Counter
cast_counts = Counter(all_casts)

# Sélectionner les 10 acteurs les plus populaires
popular_casts = [actor for actor, _ in cast_counts.most_common(10)]

# Créer une nouvelle colonne pour chaque acteur populaire
for actor in popular_casts:
    df_copy[f'ACTEUR_{actor}'] = df_copy['CAST'].apply(lambda x: 1 if actor in x else 0)


In [8]:
df_copy.columns

Index(['Unnamed: 0', 'MOVIE_NAME', 'REVIEWS', 'SYNOPSIS', 'YEAR', 'CAST',
       'DURATION_MIN', 'GENRES', 'PRODUCERS', 'RATING_SUR_5',
       'MAJORITY_SENTIMENT', 'MOST_FREQUENT_SENTIMENT', 'Rating_Category',
       'genre_War', 'genre_LGBTQ+', 'genre_Fantasy', 'genre_History',
       'genre_Holiday', 'genre_Documentary', 'genre_Mystery', 'genre_Music',
       'genre_Action', 'genre_Kids & Family', 'genre_Horror', 'genre_Anime',
       'genre_Musical', 'genre_Drama', 'genre_TV Movie', 'genre_Biography',
       'genre_Western', 'genre_Animation', 'genre_Sci-Fi', 'genre_Comedy',
       'genre_Thriller', 'genre_Adventure', 'genre_Sports', 'genre_Family',
       'genre_Crime', 'genre_Science Fiction', 'genre_Mystery & Thriller',
       'genre_Romance', 'ACTEUR_Tom Cruise', 'ACTEUR_Samuel L. Jackson',
       'ACTEUR_Mickie McGowan', 'ACTEUR_Ving Rhames', 'ACTEUR_Brad Pitt',
       'ACTEUR_Jr.', 'ACTEUR_Sherry Lynn', 'ACTEUR_Robert De Niro',
       'ACTEUR_Ralph Fiennes', 'ACTEUR_Jack Ange

meme chose pour les producers

In [9]:
# Assurez-vous que la colonne 'PRODUCERS' contient une liste
df_copy['PRODUCERS'] = df_copy['PRODUCERS'].apply(lambda x: x.split('; ') if isinstance(x, str) else [])

# Filtrer les entrées non pertinentes dans la colonne 'PRODUCERS' ('Show All…', 'Indisponible', '')
df_copy['PRODUCERS'] = df_copy['PRODUCERS'].apply(lambda x: [producer for producer in x if producer not in ['Show All…', 'Indisponible', '']])

# Filtrer les films avec une note supérieure à 3.5
good_movies = df_copy[df_copy['RATING_SUR_5'] >= 3.5]

# Aplatir la liste des producteurs pour les films ayant une note > 3.5
all_producers = [producer for sublist in good_movies['PRODUCERS'] for producer in sublist]

# Compter la fréquence de chaque producteur
producer_counts = Counter(all_producers)

# Sélectionner les 10 producteurs les plus populaires
popular_producers = [producer for producer, _ in producer_counts.most_common(10)]

# Créer une nouvelle colonne pour chaque producteur populaire
for producer in popular_producers:
    df_copy[f'PRODUCER_{producer}'] = df_copy['PRODUCERS'].apply(lambda x: 1 if producer in x else 0)

In [10]:
df_copy.columns

Index(['Unnamed: 0', 'MOVIE_NAME', 'REVIEWS', 'SYNOPSIS', 'YEAR', 'CAST',
       'DURATION_MIN', 'GENRES', 'PRODUCERS', 'RATING_SUR_5',
       'MAJORITY_SENTIMENT', 'MOST_FREQUENT_SENTIMENT', 'Rating_Category',
       'genre_War', 'genre_LGBTQ+', 'genre_Fantasy', 'genre_History',
       'genre_Holiday', 'genre_Documentary', 'genre_Mystery', 'genre_Music',
       'genre_Action', 'genre_Kids & Family', 'genre_Horror', 'genre_Anime',
       'genre_Musical', 'genre_Drama', 'genre_TV Movie', 'genre_Biography',
       'genre_Western', 'genre_Animation', 'genre_Sci-Fi', 'genre_Comedy',
       'genre_Thriller', 'genre_Adventure', 'genre_Sports', 'genre_Family',
       'genre_Crime', 'genre_Science Fiction', 'genre_Mystery & Thriller',
       'genre_Romance', 'ACTEUR_Tom Cruise', 'ACTEUR_Samuel L. Jackson',
       'ACTEUR_Mickie McGowan', 'ACTEUR_Ving Rhames', 'ACTEUR_Brad Pitt',
       'ACTEUR_Jr.', 'ACTEUR_Sherry Lynn', 'ACTEUR_Robert De Niro',
       'ACTEUR_Ralph Fiennes', 'ACTEUR_Jack Ange

In [11]:
df_copy['DURATION_MIN'] = df_copy['DURATION_MIN'].apply(pd.to_numeric, errors='coerce')  # Assurez-vous qu'il s'agit de données numériques

# Normalisation de DURATION
scaler = StandardScaler()
df_copy['DURATION_MIN'] = scaler.fit_transform(df_copy[['DURATION_MIN']])


**Vectorisation des colonnes textuelles**

In [12]:
data=df_copy

In [13]:
data_without_rating = data[data['RATING_SUR_5'].isna()]  # Films sans 'RATING'
data = data.dropna(subset=['RATING_SUR_5']) 

In [14]:
data['GENRES'] = data['GENRES'].apply(lambda x: ' '.join(x) if isinstance(x, list) else str(x))
data['CAST'] = data['CAST'].apply(lambda x: ' '.join(x) if isinstance(x, list) else str(x))
data['PRODUCERS'] = data['PRODUCERS'].apply(lambda x: ' '.join(x) if isinstance(x, list) else str(x))


In [15]:
data = data.fillna("")

# Vectorisation des colonnes textuelles
vectorizer_synopsis = TfidfVectorizer(max_features=800)
vectorizer_reviews = TfidfVectorizer(max_features=1000)
vectorizer_genres = TfidfVectorizer(max_features=500)
vectorizer_cast = TfidfVectorizer(max_features=500)
vectorizer_producers = TfidfVectorizer(max_features=500)

# Transformation des colonnes textuelles en caractéristiques TF-IDF
synopsis_features = vectorizer_synopsis.fit_transform(data['SYNOPSIS']).toarray()
reviews_features = vectorizer_reviews.fit_transform(data['REVIEWS']).toarray()
genre_features = vectorizer_genres.fit_transform(data['GENRES']).toarray()
cast_features = vectorizer_cast.fit_transform(data['CAST']).toarray()
producers_features = vectorizer_producers.fit_transform(data['PRODUCERS']).toarray()

In [16]:
# Combiner les caractéristiques textuelles
textual_features = np.hstack([synopsis_features, reviews_features, genre_features, cast_features, producers_features])


In [17]:
# Vérifier le type des données de la colonne DURATION_MIN
print(data['DURATION_MIN'].dtype)

# Vérifier si la colonne contient des chaînes de caractères
print(data['DURATION_MIN'].head())


object
0   -0.444013
1    0.956833
2   -0.163844
3    0.793401
4   -0.187191
Name: DURATION_MIN, dtype: object


In [18]:
# Traitement des colonnes numériques
# Convertir la colonne 'DURATION_MIN' en type numérique, en remplaçant les valeurs non numériques par NaN
data['DURATION_MIN'] = pd.to_numeric(data['DURATION_MIN'], errors='coerce')
data['DURATION_MIN'] = data['DURATION_MIN'].fillna(data['DURATION_MIN'].mean())
numerical_columns = ['DURATION_MIN']
numerical_data = data[numerical_columns].values
data[numerical_columns] = data[numerical_columns].apply(pd.to_numeric, errors='coerce')
scaler = StandardScaler()
numerical_features = scaler.fit_transform(numerical_data)  # Normalisation des colonnes numériques


In [19]:
# Colonnes binaires/one-hot (les genres , acteurs et producteurs déjà sous forme binaire)
binary_columns = [col for col in data.columns if col.startswith('genre_') or col.startswith('ACTEUR_') or col.startswith('PRODUCER_')]
binary_data = data[binary_columns].values

In [20]:
from sklearn.preprocessing import OneHotEncoder
categorical_columns = ['MOST_FREQUENT_SENTIMENT', 'Rating_Category']
encoder = OneHotEncoder()
categorical_data = encoder.fit_transform(data[categorical_columns])


In [21]:
X = np.hstack([textual_features, numerical_features, binary_data, categorical_data.toarray()])
y = data['RATING_SUR_5']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)


In [22]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Ajouter HistGradientBoostingRegressor à la liste des modèles
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest Regressor': RandomForestRegressor(),
    'Support Vector Regressor': SVR(),
    'HistGradientBoosting Regressor': HistGradientBoostingRegressor()
}

# Évaluation des modèles avec validation croisée (scoring : 'neg_mean_absolute_error' pour la régression)
results = {}
for name, model in models.items():
    # Validation croisée (ici 5-fold)
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')
    results[name] = -cv_scores.mean()  # Négatif pour avoir des valeurs positives

# Affichage des résultats de validation croisée
print("Scores de validation croisée par modèle :")
for name, score in results.items():
    print(f"{name}: {score:.4f}")

# Choisir le meilleur modèle
best_model_name = min(results, key=results.get)  # Minimiser l'erreur
best_model = models[best_model_name]
print(f"\nLe meilleur modèle est : {best_model_name}")

# Entraîner le meilleur modèle sur l'ensemble d'entraînement
best_model.fit(X_train, y_train)

# Prédictions sur l'ensemble de test
y_pred = best_model.predict(X_test)

# Calcul des métriques de performance
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print("\nMétriques de régression :")
print(f"Mean Absolute Error (MAE) : {mae:.4f}")
print(f"Mean Squared Error (MSE) : {mse:.4f}")
print(f"Root Mean Squared Error (RMSE) : {rmse:.4f}")


Scores de validation croisée par modèle :
Linear Regression: 0.7980
Random Forest Regressor: 0.2167
Support Vector Regressor: 0.2369
HistGradientBoosting Regressor: 0.2152

Le meilleur modèle est : HistGradientBoosting Regressor

Métriques de régression :
Mean Absolute Error (MAE) : 0.2198
Mean Squared Error (MSE) : 0.0997
Root Mean Squared Error (RMSE) : 0.3158


In [23]:
import joblib

# Chemin pour sauvegarder le modèle
model_path = r"C:\Users\Aycha\Desktop\M2_BDIA\NLP\Projet_movie\final\model\best_model_0.09.pkl"

# Sauvegarder le modèle avec joblib
joblib.dump(best_model, model_path)
print(f"Modèle sauvegardé sous {model_path}")

Modèle sauvegardé sous C:\Users\Aycha\Desktop\M2_BDIA\NLP\Projet_movie\final\model\best_model_0.09.pkl


In [24]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_iter': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}

grid_search = GridSearchCV(HistGradientBoostingRegressor(), param_grid, cv=5, scoring='neg_mean_absolute_error')
grid_search.fit(X_train, y_train)
print("Meilleurs hyperparamètres : ", grid_search.best_params_)


Meilleurs hyperparamètres :  {'learning_rate': 0.1, 'max_depth': 7, 'max_iter': 100}


In [26]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# Initialisation du modèle avec les meilleurs hyperparamètres
model = HistGradientBoostingRegressor(
    learning_rate=0.1, 
    max_depth=7, 
    max_iter=100
)

# Entraînement du modèle sur l'ensemble d'entraînement
model.fit(X_train, y_train)

# Prédictions sur l'ensemble de test
y_pred = model.predict(X_test)

# Calcul des métriques de performance
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
# Affichage des métriques
print("\nMétriques de régression :")
print(f"Mean Absolute Error (MAE) : {mae:.4f}")
print(f"Mean Squared Error (MSE) : {mse:.4f}")
print(f"Root Mean Squared Error (RMSE) : {rmse:.4f}")
print(f'R² Score: {r2:.4f}')



Métriques de régression :
Mean Absolute Error (MAE) : 0.2211
Mean Squared Error (MSE) : 0.0994
Root Mean Squared Error (RMSE) : 0.3154
R² Score: 0.8484
